### Please set the environment

In [1]:
environment = "taxi" # among random maze, cliff or taxi

if environment == "taxi":
    from taxi.training_utils import *
    from taxi.learning_algorithms import *
    env = gym.make("Taxi-v3")
if environment == "maze":
    from mazes.training_utils import *
    from mazes.learning_algorithms import *
    from environment.random_maze import *
    gym.envs.register(id=env_id, entry_point=RandomMaze, max_episode_steps=200)
    env = gym.make(env_id)
    
if environment == "cliff":
    from mazes.training_utils import *
    from mazes.learning_algorithms import *
    from environments.cliff import *
    gym.envs.register(id=env_id, entry_point=RandomCliff, max_episode_steps=200)
    env = gym.make(env_id)

import pickle

# Load the autoreload extension
%load_ext autoreload
%autoreload 2

In [2]:
train = True
load = not train

test_freq=50
num_episodes=2000

In [3]:
if train:
    num_avg = 1
    stats = {"SCRN": {}, "SPG": {}, "SPG Entropy": {}}
    for i in range(num_avg):
        print(f"========== TRAINING RUN {i} OUT OF {num_avg} ===========")
        print("********** TRAINING WITH SCRN **********")
        stats_SCRN = discrete_SCRN(env, num_episodes=num_episodes, test_freq=test_freq)
        stats["SCRN"].update({i: stats_SCRN})
        print("********** TRAINING WITH SPG ********")
        stats_DPG = discrete_policy_gradient(env, num_episodes=num_episodes, test_freq=test_freq)
        stats["SPG"].update({i: stats_DPG})
        print("********** TRAINING WITH regularized SPG ********")
        stats_DPG = discrete_policy_gradient(env, entropy_bonus=True, num_episodes=num_episodes, test_freq=test_freq)
        stats["SPG Entropy"].update({i: stats_DPG})
else:
    with open("results.pkl", "rb") as f:
        average_stats = pickle.load(f)

========== TRAINING RUN 0 OUT OF 1 ===========
********** TRAINING WITH SCRN **********
====== Testing the current policy ======
==== Testing episode 0 ====
==== Testing episode 1 ====
==== Testing episode 2 ====
==== Testing episode 3 ====
==== Testing episode 4 ====
==== Testing episode 5 ====
==== Testing episode 6 ====
==== Testing episode 7 ====
==== Testing episode 8 ====
==== Testing episode 9 ====
==== Testing episode 10 ====
==== Testing episode 11 ====
==== Testing episode 12 ====
==== Testing episode 13 ====
==== Testing episode 14 ====
==== Testing episode 15 ====
==== Testing episode 16 ====
==== Testing episode 17 ====
==== Testing episode 18 ====
==== Testing episode 19 ====
==== Testing episode 20 ====
==== Testing episode 21 ====
==== Testing episode 22 ====
==== Testing episode 23 ====
==== Testing episode 24 ====
==== Testing episode 25 ====
==== Testing episode 26 ====
==== Testing episode 27 ====
==== Testing episode 28 ====
==== Testing episode 29 ====
==== Testin

KeyboardInterrupt: 

In [ ]:
average_stats = {"SCRN": {}, "SPG": {}, "SPG Entropy": {}}
std_stats = {"SCRN": {}, "SPG": {}, "SPG Entropy": {}}
average_stats["SCRN"] = {key: np.median([stats["SCRN"][i][key] for i in range(num_avg)], axis=0)
                         for key in ["good_policy", "taus", "steps", "rewards", "optimum", "history_probs"]}
average_stats["SPG"] = {key: np.median([stats["SPG"][i][key] for i in range(num_avg)], axis=0)
                         for key in ["good_policy", "taus", "steps", "rewards", "optimum", "history_probs"]}
average_stats["SPG Entropy"] = {key: np.median([stats["SPG Entropy"][i][key] for i in range(num_avg)], axis=0)
                         for key in ["good_policy", "taus", "steps", "rewards", "optimum", "history_probs"]}
std_stats["SCRN"] = {key: np.std([stats["SCRN"][i][key] for i in range(num_avg)], axis=0) / np.sqrt(num_avg)
                         for key in ["taus", "steps", "rewards"]}
std_stats["SPG"] = {key: np.median([stats["SPG"][i][key] for i in range(num_avg)], axis=0) / np.sqrt(num_avg)
                         for key in ["taus", "steps", "rewards"]}
std_stats["SPG Entropy"] = {key: np.median([stats["SPG Entropy"][i][key] for i in range(num_avg)], axis=0) / np.sqrt(num_avg)
                         for key in ["taus", "steps", "rewards"]}

In [ ]:
show_SCRN = True
show_SPG = True

for key in average_stats.keys():
    print(key, average_stats[key]["good_policy"])

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.figure()
plt.figure(figsize=(6, 4))
plt.plot(average_stats["SCRN"]["steps"], label="SCRN")
plt.plot(average_stats["SPG"]["steps"], label="SPG")
plt.plot(average_stats["SPG Entropy"]["steps"], label="SPG Entropy")
plt.ylabel(r"Steps needed")
plt.xlabel("Episodes")
plt.legend(loc="best")
plt.show()

In [ ]:
plt.figure()
QOI_SCRN = average_stats["SCRN"]["taus"]
STD_SCRN = std_stats["SCRN"]["taus"]
QOI_SPG = average_stats["SPG"]["taus"]
STD_SPG = std_stats["SPG"]["taus"]
QOI_ESPG = average_stats["SPG Entropy"]["taus"]
STD_ESPG = std_stats["SPG Entropy"]["taus"]
plt.plot(np.arange(0, num_episodes, step=test_freq), QOI_SCRN, label="SCRN")
plt.fill_between(np.arange(0, num_episodes, step=test_freq), QOI_SCRN-STD_SCRN, QOI_SCRN+STD_SCRN, alpha=0.3)
plt.plot(np.arange(0, num_episodes, step=test_freq), QOI_SPG, label="SPG")
plt.fill_between(np.arange(0, num_episodes, step=test_freq), QOI_SPG-STD_SPG, QOI_SPG+STD_SPG, alpha=0.3)
plt.plot(np.arange(0, num_episodes, step=test_freq), QOI_ESPG, label="SPG Entropy")
plt.fill_between(np.arange(0, num_episodes, step=test_freq), QOI_ESPG-STD_ESPG, QOI_ESPG+STD_ESPG, alpha=0.3)
plt.legend(loc="best")
plt.xlabel("Testing policy episodes")
plt.ylabel(r"$\frac{J(\theta^{*}) - J(\theta)}{\vert \vert \nabla J(\theta) \vert \vert}$")
plt.show()

In [ ]:
plt.figure()
plt.plot(average_stats["SCRN"]["rewards"], label="SCRN")
plt.plot(average_stats["SPG"]["rewards"], label="SPG")
plt.plot(average_stats["SPG Entropy"]["rewards"], label="SPG Entropy")
plt.legend(loc="best")
plt.xlabel("Episodes")
plt.ylabel("Rewards during training")
plt.show()